In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


from tensorflow.keras.models import Sequential
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Activation
from tensorflow.keras import backend as K

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [0]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

print(x_train.shape)
print(x_test.shape)

In [0]:
# Define the text labels
fashion_mnist_labels = ["T-shirt/top",  # index 0
                        "Trouser",      # index 1
                        "Pullover",     # index 2 
                        "Dress",        # index 3 
                        "Coat",         # index 4
                        "Sandal",       # index 5
                        "Shirt",        # index 6 
                        "Sneaker",      # index 7 
                        "Bag",          # index 8 
                        "Ankle boot"]   # index 9
num_class = 10

In [0]:
fig, ax = plt.subplots(2,10,figsize=(15,4.5))
fig.subplots_adjust(hspace=0.1, wspace=0.1)
for j in range(0,2):
    for i in range(0, 10):
        ax[j,i].imshow(x_train[i+j*10], cmap='gray')
        ax[j,i].set_title(fashion_mnist_labels[y_train[i+j*10]])
        ax[j,i].axis('off')
plt.show()

In [0]:
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)

n_input = x_train.shape[1]

In [0]:
encoding_dim = 500 

input_img = Input(shape=(n_input,))

encoded = Dense(encoding_dim, activation='sigmoid')(input_img)  # layer 1

encoding_dim2 = 200

encoded2 = Dense(encoding_dim2, activation='sigmoid')(encoded)

decoded2 = Dense(n_input, activation='sigmoid')(encoded2)

# satukan model

autoencoder = Model(input_img, decoded2)

In [0]:
autoencoder.compile(optimizer='rmsprop', loss='binary_crossentropy')
autoencoder.fit(x_train, x_train,
                epochs=40,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

In [0]:
inputs2 = Input(shape=(n_input,))
x = encoder(inputs2)
outputs2 = Dense(num_class, activation='softmax')(x)

pretrain = Model(inputs2, outputs2)
pretrain.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

pretrain.fit(x_train, y_train,
                epochs=20,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, y_test))

In [0]:
inputs = Input(shape=(n_input,))
x = Dense(500, activation='sigmoid')(inputs)
x = Dense(200, activation='sigmoid')(x)
outputs = Dense(num_class, activation='softmax')(x)

shallow = Model(inputs, outputs)
shallow.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

shallow.fit(x_train, y_train,
                epochs=20,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, y_test))

In [0]:

y_pretrain = np.argmax(pretrain.predict(x_test),axis=1)
y_shallow = np.argmax(shallow.predict(x_test),axis=1)


print('Layer-wise Pretrain Performance')
print('Confusion Matrix')
print(confusion_matrix(np.argmax(y_test,axis=1), y_pretrain))
print()

print('Accuracy =', accuracy_score(np.argmax(y_test,axis=1), y_pretrain))
print()

print('Shallow Performance')
print('Confusion Matrix')
print(confusion_matrix(np.argmax(y_test,axis=1), y_shallow))
print()

print('Accuracy =', accuracy_score(np.argmax(y_test,axis=1), y_shallow))
print()
